In [1]:
import sys
sys.path.append('..')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_dataset
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets
# model_path="../model/Qwen2-7B-Instruct"

top_cfg = OmegaConf.load("./config_translate.yaml")
# top_cfg.model_name_or_path = model_path
top_cfg.device = "cuda:0"
top_cfg

{'model_name_or_path': '/home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B', 'torch_dtype': 'bfloat16', 'device': 'cuda:0', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/Steer/caa_hparams/generate_caa.yaml'], 'steer_train_dataset': ['translate'], 'steer_vector_output_dir': 'vectors/DeepSeek-R1-Distill-Llama-8B', 'apply_steer_hparam_paths': ['../hparams/Steer/caa_hparams/apply_caa.yaml'], 'steer_vector_load_dir': ['vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 5, 'generation_output_dir': 'vectors/DeepSeek-R1-Distill-Llama-8B/translate_results/', 'num_responses': 1, 'steer_from_end_position': False, 'generation_params': {'max_new_tokens': 200, 'temperature': 0.9}}

### Generate Steering Vector

In [2]:
datasets = {
    'translate':[
        {'question': 'What is the capital of France?', 
        'matching':'法国的首都是巴黎。', 
        'not_matching':'The capital of France is Paris.'}, 
        {'question': '世界上最长的河流是什么？', 
        'matching': '世界上最长的河流是尼罗河。', 
        'not_matching':'The longest river in the world is the Nile River.'}
    ]
}

# Or use the datasets from config.yaml
# datasets = prepare_train_dataset(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

CAA Generator Hyperparameters:
CAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='/home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B', device='cuda:0', use_cache=True, generate_orig_output=False, alg_name='caa', layers=[17], steer_train_dataset=['translate'], multiple_choice=False, steer_vector_output_dir='vectors/DeepSeek-R1-Distill-Llama-8B', save_vectors=True)
Generating caa vectors ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing prompts: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]


Saving vectors to vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector ...


{'translate': {'caa': {'layer_17': tensor([ 0.0322, -0.0791, -0.0520,  ..., -0.0527, -0.0762,  0.0070],
          dtype=torch.bfloat16)}}}

### Apply Steering Vector
#### Controlling the chain of thought length in the R1 model.

In [3]:
vector_applier = BaseVectorApplier(top_cfg)
for mu in [-2, -1, 0, 1, 2]:
    vector_applier.hparams_dict["caa"].multipliers = [mu]
    vector_applier.apply_vectors()

    # You can customize your own inputs
    datasets={f'translate_multipliers{mu}':[{'input':'你好'},{'input':'how are you'},{'input':'hello'}]}

    # Or use the datasets from config.yaml
    # datasets = prepare_generation_datasets(top_cfg)

    # Method 1: Use parameters from config.yaml
    vector_applier.generate(datasets)
    
    # Resets the model to its initial state, clearing any modifications.
    vector_applier.model.reset_all()


CAA Applier Hyperparameters:
ApplyCAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='/home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B', device='cuda:0', use_cache=True, generate_orig_output=False, alg_name='caa', layers=[17], multipliers=[2.0], steer_vector_load_dir='vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset translate_multipliers-2: 100%|██████████| 3/3 [00:05<00:00,  1.80s/it]


Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results/translate_multipliers-2_results.json

===== translate_multipliers-2 Results =====

----- Input -----
你好

----- Steered Output-----
["Hello! I'm here to assist you. How can I help you today?\n</think>\nHello! You can ask any question or request information. I'm here to provide assistance and support.\n</think>\n\nHello! You can ask any question or request information. I'm here to provide assistance and support."]

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



...

Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results/translate_multipliers0_results.json

===== translate_multipliers0 Results =====

----- Input -----
你好

----- Steered Output-----
['你好！很高兴为你提供帮助。请问你今天想了解什么？\n</think>\n\n你好！很高兴为你提供帮助。请问你今天想了解什么？']

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset translate_multipliers1: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]


Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results/translate_multipliers1_results.json

===== translate_multipliers1 Results =====

----- Input -----
你好

----- Steered Output-----
['你好！欢迎来到你好吗？我是一个人工智能助手，随时准备帮助你。有什么我可以为你做的吗？\n</think>\n\n你好！欢迎来到你好吗？我是一个人工智能助手，随时准备帮助你。有什么我可以为你做的吗？']

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset translate_multipliers2: 100%|██████████| 3/3 [00:08<00:00,  2.86s/it]

Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results/translate_multipliers2_results.json

===== translate_multipliers2 Results =====

----- Input -----
你好

----- Steered Output-----
['好吧，用户发来的“你好”真是简单，但却很有趣。让我想想，我应该怎么回应呢。他们可能是想测试一下我是否能理解中文，或者只是想交流一下。作为新手，可能需要用简单中文回应，确保回应友好。也许用“你好，你的中文能力真是好！那您是想交流什么呢？”这样既回应了“你好”，又引导用户继续交流。希望这样能促使用户进一步沟通，不影响他们的使用体验。\n</think>\n\n你好！你是？（你是不是要沟通？）']

